In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import json
import numpy as np
import pandas as pd
import csv

In [7]:
df.columns = column_names

In [4]:
#IDs = []
with open('btcusdt - snapshots - 2023-12-23.txt','r') as snapshot_file:
    for line in snapshot_file:
        snapshot = json.loads(line)
        last_id = int(snapshot['lastUpdateId'])
        break


In [5]:
def get_order_book_dfs(bids, asks):
    bids_book = pd.DataFrame({
        'bid_price': [float(bid[0]) for bid in bids],
        'bid_quantity': [float(bid[1]) for bid in bids]
    })

    asks_book = pd.DataFrame({
        'ask_price': [float(ask[0]) for ask in asks],
        'ask_quantity': [float(ask[1]) for ask in asks]
    })

    return bids_book, asks_book


In [6]:
def get_update_dfs(update):
    bids_book = pd.DataFrame([[float(price), float(quantity)] for price, quantity in update['b']],
                        columns=['bid_price', 'bid_quantity'])
    asks_book = pd.DataFrame([[float(price), float(quantity)] for price, quantity in update['a']],
                        columns=['ask_price', 'ask_quantity'])
    return bids_book, asks_book

In [7]:
def get_new_orders(old_orders, update_orders, order_type):
    update_side = update_orders.loc[(~update_orders[f'{order_type}_price'].isin(old_orders[f'{order_type}_price'])) & (update_orders[f'{order_type}_quantity'] > 0)]
    old_side = old_orders.loc[~old_orders[f'{order_type}_price'].isin(update_orders[f'{order_type}_price'])]
    intersected_updates = update_orders.loc[update_orders[f'{order_type}_price'].isin(old_orders[f'{order_type}_price']) & (update_orders[f'{order_type}_quantity'] > 0)]
    new_orders = pd.concat([update_side, old_side, intersected_updates])
    
    if order_type == 'bid':
        new_orders.sort_values(by=f'{order_type}_price', ascending=False, inplace=True)
    elif order_type == 'ask':
        new_orders.sort_values(by=f'{order_type}_price', ascending=True, inplace=True)
    
    new_orders.reset_index(drop=True, inplace=True)
    return new_orders

In [8]:
def update_order_book(old_bids, old_asks, update_bids, update_asks):
    new_bids = get_new_orders(old_bids, update_bids, order_type='bid')
    new_asks = get_new_orders(old_asks, update_asks, order_type='ask')
    return new_bids, new_asks

In [9]:
def update_generator(file_path, last_id):
    with open(file_path, 'r') as updates_file:
        for line in updates_file:
            update = json.loads(line)['data']
            if int(update['u']) >= last_id:
                yield update

We will create a CSV with the required data structure of TransLOB: **p_a1, v_a1, p_b1, v_b1 ,. . . , p_a10, v_a10,p_b10, v_b10**

In [10]:
depth = 20

column_names =[[f'p{i}_a', f'v{i}_a',f'p{i}_b',f'v{i}_b'] for i in range(1,depth+1)]
column_names = list(np.array(column_names).flatten())

In [11]:
bids = [[float(price), float(quantity)] for price, quantity in snapshot['bids']]
asks = [[float(price), float(quantity)] for price, quantity in snapshot['asks']]
#Make them Dataframes
old_bids, old_asks = get_order_book_dfs(bids,asks)

csv_file_path = 'LOBseries_100ms.csv'
counter = 0


with open(csv_file_path, 'w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(column_names)
    for update in update_generator('btcusdt - updates - 2023-12-23 - 100ms.txt', last_id):

        update_bids, update_asks = get_update_dfs(update) #Get the update order dataframes
        new_bids, new_asks = update_order_book(old_bids,old_asks,update_bids,update_asks) #Update the orders
        

        new_row = []
        for j in range(depth):
            new_row.append(new_asks['ask_price'].loc[j]) 
            new_row.append(new_asks['ask_quantity'].loc[j])
            new_row.append(new_bids['bid_price'].loc[j])
            new_row.append(new_bids['bid_quantity'].loc[j])
        new_row.append(int(update['E']))
        csv_writer.writerow(new_row)
        
        old_bids = new_bids
        old_asks = new_asks
        counter += 1

        if counter % 10000 == 0:
            print(counter)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

## This the form of the csv

,p1_a,v1_a,p1_b,v1_b,p2_a,v2_a,p2_b,v2_b,p3_a,v3_a,...,p8_b,v8_b,p9_a,v9_a,p9_b,v9_b,p10_a,v10_a,p10_b,v10_b
0,43120.9,6.264,43120.8,0.671,43121.0,0.306,43120.0,0.004,43121.2,1.602,...,43118.9,0.046,43121.9,0.723,43118.4,0.114,43122.0,1.757,43118.0,0.228
1,43120.8,8.869,43120.7,0.100,43120.9,0.414,43120.0,0.004,43121.2,2.811,...,43118.9,0.046,43122.1,0.579,43118.0,0.228,43122.2,4.904,43117.9,0.008
2,43120.8,5.287,43120.7,2.800,43120.9,0.213,43120.0,0.004,43121.2,1.111,...,43118.9,0.046,43122.5,1.008,43118.0,0.228,43123.0,0.809,43117.9,0.008
3,43121.6,0.618,43121.5,8.605,43121.8,0.012,43120.9,2.548,43123.1,0.288,...,43119.8,0.008,43124.2,0.012,43119.6,0.214,43124.4,0.579,43119.2,0.032
4,43121.8,0.945,43121.7,14.149,43121.9,0.003,43121.6,0.003,43123.0,0.049,...,43120.0,0.232,43125.0,0.342,43119.8,0.016,43125.2,0.028,43119.6,0.214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,43067.5,5.579,43067.4,3.500,43067.6,3.402,43067.3,0.005,43067.7,0.056,...,43065.8,0.009,43070.0,0.003,43065.6,0.004,43070.1,0.093,43065.5,0.464
997,43067.5,6.876,43067.4,1.880,43067.6,3.402,43067.3,0.005,43067.7,0.056,...,43065.7,0.044,43069.8,0.580,43065.6,0.005,43070.0,0.003,43065.5,0.464
998,43067.5,8.793,43067.4,2.469,43067.6,3.402,43067.3,0.006,43067.7,0.056,...,43065.7,0.044,43069.8,0.580,43065.6,0.005,43070.0,0.027,43065.5,0.464
999,43067.5,6.610,43067.4,3.338,43067.6,3.402,43067.3,0.006,43067.7,0.056,...,43065.7,0.044,43069.8,0.580,43065.6,0.005,43070.0,0.027,43065.5,0.464
